In [1]:
from lib import data_loaders
from lib import dendra

from dotenv import load_dotenv
import os
import requests
import pandas as pd

In [2]:
load_dotenv()
dendra_pass = os.getenv("DENDRA_PASS")
Dendra = data_loaders.Dendra(email="dragar@lynker.com", password=dendra_pass)

In [6]:
# parameters: start and end time
# Note the queries default to local time. Add a captial 'Z' to the end of the timestamp to indicate UTC and many functions have a parameter for local vs utc
begins_at = "2023-01-01T00:00:00"
ends_before = (
    dendra.time_format()
)  # time_format without argument gives current datetime. #'2020-03-01T00:00:00'

In [25]:
measurement = "WellDepth"
# measurement = "AirTemperature"
query_refinement = {"is_hidden": False}
measurement_list = []  # list of only datastreams that you wish to download data from

ds_list = Dendra.list_datastreams_by_measurement(
    measurement, "", [], "tnc", query_refinement
)

for ds in ds_list:
    dsm = Dendra.get_meta_datastream_by_id(
        ds["_id"]
    )  # This will pull full datastream metadata in JSON format
    station_name = dsm["station_lookup"]["name"]
    print(station_name, ds["name"], ds["_id"])
    measurement_list.append(ds["_id"])

Dangermond Escondido 5 Depth to Groundwater 6511f67d2148db626248533e
Dangermond Buckhorn 2 Depth to Groundwater 6511f7e82148db3a1e485341
Dangermond Alegria Depth to Groundwater 6511f7e85526453a3c3edcd7
Dangermond Alexander Ramajal Depth to Groundwater 6511f7e8808187594cc93c0b
Dangermond Buckhorn 1 Depth to Groundwater 6511f7e8d070873fc4d574e2
Dangermond East Ramajal Depth to Groundwater 6511f7e90209692d22a0add0
Dangermond Diamond Corral Depth to Groundwater 6511f7e95526459d613edcd9
Dangermond Damsite Canyon Depth to Groundwater 6511f7e95c0d5fe521698416
Dangermond Cojo Canyon Depth to Groundwater 6511f7e9f1438dfeb44e11a9
Dangermond Escondido 3 Depth to Groundwater 6511f7ea55264589773edcdb
Dangermond Escondido 1 Depth to Groundwater 6511f7ea5c0d5f9de3698418
Dangermond Escondido 2 Depth to Groundwater 6511f7ea8081871abec93c0d
Dangermond Escondido 3B Depth to Groundwater 6511f7ea8081871bfbc93c0f
Dangermond Escondido 4 Depth to Groundwater 6511f7ead07087d401d574e5
Dangermond Gaspar 2 Depth 

In [30]:
# request data for single datastream ID
res = Dendra.get_datapoints(
    begins_at="2023-01-01T00:00:00",
    datastream_id=measurement_list[53],
    ends_before="2024-05-01T00:00:00",
    name="other",
)

In [31]:
res

,timestamp_utc,SciMainRanchNorthWell_Well_Water_Depth
timestamp_local,,
2023-01-01 00:00:00,2023-01-01 08:00:00+00:00,-26.276033
2023-01-01 00:05:00,2023-01-01 08:05:00+00:00,-26.273033
2023-01-01 00:10:00,2023-01-01 08:10:00+00:00,-26.274033
2023-01-01 00:15:00,2023-01-01 08:15:00+00:00,-26.272033
2023-01-01 00:20:00,2023-01-01 08:20:00+00:00,-26.269033
...,...,...
2024-04-30 23:30:00,2024-05-01 07:30:00+00:00,-9.705033
2024-04-30 23:35:00,2024-05-01 07:35:00+00:00,-9.706033
2024-04-30 23:45:00,2024-05-01 07:45:00+00:00,-9.705033
